In [1]:
# PART OF THIS CODE IS FROM MICHAEL MURPHY - THANKS!
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from collections import OrderedDict
import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
import glob, re
import seaborn as sns
pd.set_option('display.max_rows', 500) 
pd.set_option('display.max_columns', 100)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.utils.multiclass import type_of_target # used to check the Y labels are appropriate for classification
from sklearn.metrics import roc_curve
from sklearn import metrics
from sklearn.utils import shuffle
from scipy import interp

In [2]:
#### Use this if DOING a fresh modeling fitting analysis
bn = False # use the percentile normalized data or no? 
log = False
stand_scaler = False
reanalysis = True
model = 'log_reg' #log_reg, rf or svm

if reanalysis:
    pre_norm_ds = [ 'plasmaall_author',
                    'urineall_author',
                    'm_oxylipin_chronic_hep_b',
                    'm_chronic_hep_b_POS',
                    'm_chronic_hep_b_NEG',
                    'm_CER_mass_spectrometry_v4',
                    'm_CER_mass_spectrometry_v4_3_CS',
                    'm_CER_mass_spectrometry_v4_0_NS',
                    'm_CER_mass_spectrometry_v4_2_FS',
                    'm_CER_mass_spectrometry_v4_1_COPD',
                    'm_EICO_mass_spectrometry_v4',
                    'm_EICO_mass_spectrometry_v4_3_CS',
                    'm_EICO_mass_spectrometry_v4_0_NS',
                    'm_EICO_mass_spectrometry_v4_2_FS',
                    'm_EICO_mass_spectrometry_v4_1_COPD',
                    'AN000580',
                    'AN000581',
                    'AN001503']

    if bn:
        path = './bn_pickles/*.pkl'
#         path = './bn_pickles/MTBLS72*.pkl'
    else:
        path = './pickles/*.pkl'

    datasets = OrderedDict()
    for fn in sorted(glob.glob(path)):
        data = pd.read_pickle(open(fn,'rb'))
        datasets[data[0]['study']] = data
    
else:
    #### Use this if NOT doing a fresh modeling fitting analysis
    pickle_file = './YES_bn_ds_models_and_sigfeat_NO_log_NO_standscal_NO_multi_mapped_labels.pkl'
    ### The non-batch corrected pickle for the dataset
    # pickle_file = './NO_bn_dataset_models_and_sigfeat_YES_log.pkl'
    datasets = pickle.load(open(pickle_file, 'rb'))

In [5]:
# This cell is for finding which metabolites can be used for the matching scripts
mz_labels = ['mz', 'Mass-to-Charge', 'mass_to_charge', 'Mass', 'mzmed', 'm/z', 'moverz_quant',
             'quant mz', 'quantified m/z', 'row m/z', 'Quant mx', 'Quantified m/z']
skip_ds = ['m_oxylipin_chronic_hep_b', 'm_GC_nmfi_and_bsi_diagnosis_v2_maf',
           'm_CER_mass_spectrometry_v4_3_CS', 'm_CER_mass_spectrometry_v4_0_NS', 
           'm_CER_mass_spectrometry_v4_2_FS', 'm_CER_mass_spectrometry_v4_1_COPD', 'm_EICO_mass_spectrometry_v4_3_CS',
           'm_EICO_mass_spectrometry_v4_3_CS','m_EICO_mass_spectrometry_v4_0_NS', 'm_EICO_mass_spectrometry_v4_2_FS',
           'm_EICO_mass_spectrometry_v4_1_COPD','m_SHOT_mass_spectrometry_v4_3_CS','m_SHOT_mass_spectrometry_v4_0_NS',
           'm_SHOT_mass_spectrometry_v4_2_FS', 'm_SHOT_mass_spectrometry_v4_1_COPD', 'm_TAG_mass_spectrometry_v4_3_CS',
           'm_TAG_mass_spectrometry_v4_0_NS', 'm_TAG_mass_spectrometry_v4_2_FS', 'm_TAG_mass_spectrometry_v4_1_COPD', 
           'm_typhoid_carriage_metabolite_profiling_mass_spectrometry_v2_maf', 'AN000452_0_Healthy',
           'AN000452_1_CRC', 'AN000452_2_Polyp', 'AN000526_0_Control', 'AN000526_2_FSGS', 'AN000526_1_MCD', 
           'AN000580', 'AN000581', 'AN000582', 'AN000583', 'AN000705', 'AN000706', 'AN000931',
           'm_CER_mass_spectrometry_v4_COPD_FS', 'm_CER_mass_spectrometry_v4_COPD_CS', 'm_CER_mass_spectrometry_v4_COPD_NS', 'm_CER_mass_spectrometry_v4_FS_CS', 
           'm_CER_mass_spectrometry_v4_FS_NS', 'm_CER_mass_spectrometry_v4_CS_NS', 'm_EICO_mass_spectrometry_v4_COPD_FS', 'm_EICO_mass_spectrometry_v4_COPD_CS',
           'm_EICO_mass_spectrometry_v4_COPD_NS', 'm_EICO_mass_spectrometry_v4_FS_CS', 'm_EICO_mass_spectrometry_v4_FS_NS', 'm_EICO_mass_spectrometry_v4_CS_NS',
           'm_SHOT_mass_spectrometry_v4_COPD_FS', 'm_SHOT_mass_spectrometry_v4_COPD_CS', 'm_SHOT_mass_spectrometry_v4_COPD_NS', 'm_SHOT_mass_spectrometry_v4_FS_CS',
           'm_SHOT_mass_spectrometry_v4_FS_NS', 'm_SHOT_mass_spectrometry_v4_CS_NS', 'm_TAG_mass_spectrometry_v4_COPD_FS', 'm_TAG_mass_spectrometry_v4_COPD_CS',
           'm_TAG_mass_spectrometry_v4_COPD_NS', 'm_TAG_mass_spectrometry_v4_FS_CS', 'm_TAG_mass_spectrometry_v4_FS_NS', 'm_TAG_mass_spectrometry_v4_CS_NS',
           'AN000076_CN_MCI', 'AN000076_CN_AD', 'AN000076_MCI_AD', 'AN000077_CN_MCI', 'AN000077_CN_AD', 'AN000077_MCI_AD', 'AN000078_CN_MCI', 'AN000078_CN_AD', 
           'AN000078_MCI_AD', 'AN000079_CN_MCI', 'AN000079_CN_AD', 'AN000079_MCI_AD', 'AN000452_Healthy_CRC', 'AN000452_Healthy_Polyp', 'AN000452_CRC_Polyp',
           'AN000525_MCD_FSGS', 'AN000525_MCD_Control', 'AN000525_FSGS_Control', 'AN000526_MCD_FSGS', 'AN000526_MCD_Control', 'AN000526_FSGS_Control']
low_res = ['IPO_aligned_MTBLS105_qMS', 'IPO_aligned_MTBLS105_SIM-MS', 'IPO_aligned_MTBLS315_mzData', 'AN000100', 'AN000101',
           'XCMS-Report-annotated-SingleClass-GCTOF.', 'XCMS-Report-annotated-SingleClass-GCTOF.plasma', 'AN000615', 'IPO_aligned_ST000381_pos', 'AN000618',
           'AN000603_plasma', 'AN000603_serum', 'AN000620_plasma', 'AN000620_serum', 'IPO_aligned_ST000385_adc2_plasma',
           'IPO_aligned_ST000385_adc2_serum', 'IPO_aligned_ST000385_adc1_plasma', 'IPO_aligned_ST000385_adc1_serum',
           'AN000625_GC', 'IPO_aligned_ST000388_GC', 'AN000628_plasma', 'AN000628_serum', 'IPO_aligned_ST000392_plasma',
           'IPO_aligned_ST000392_serum', 'AN000633', 'IPO_aligned_ST000396', 'AN001390all_author', 
           'IPO_aligned_ST000865_batch2_raw', 'IPO_aligned_ST000865_batch3_raw', 'IPO_aligned_ST000865_onebatch']

def parse_MTBLS352(datasets):
    '''
    Input: the full dataset object
    '''
    for ds in datasets['MTBLS352']:
        if 'DEMO' in ds['data_set']:
            data = list(ds['peaks'])
            rt = [ele.split('_')[0] for ele in data]
            mz = [ele.split('_')[1][:-3] for ele in data]
            peaks = pd.DataFrame({'rt': rt, 'mz':mz})
            ds['peaks'] = peaks
    return datasets

# turn the following off if run once!
# datasets = parse_MTBLS352(datasets) 
        
for k,v in datasets.items():
    print(k)
    string = ''
#     if k != 'MTBLS124':
#         continue
    for ds in v:
        print(ds['data_set'],ds['peaks'].head())
#         string = string + "'"+ds['data_set']+ "', "
#     print(string)
# 
#         print(ds['data_set'],list(ds['peaks']))




#         if ds['data_set'] in skip_ds:
#             continue
#         if ds['data_set'] in low_res:
#             continue

#         for header in list(ds['peaks']):
#             if header in mz_labels:
#                 mz_data = ds['peaks'][header]
# #                 print(ds['data_set'], ds['peaks'][header].head())
#                 break
    
    
    
    
    
#         print(mz_data.shape)
#         p_mask = ds['pvalues'] < 0.05
#         mz_data = mz_data[p_mask]
        
#         coef_mask = ds['clf'].coef_.squeeze() != 0
#         mz_data = mz_data[coef_mask]
#         print(mz_data.shape)
                
# (ST000329) is kinda fucked ..check it out ~ cant do anything for the non reprocessed data...
# (ST000355) (nothing) / 356 (nothing) / 381(quant mz)/ 
# 383 (quantified m/z) / 385 (Quant mx) / 388 (row m/z) / 389 (quantified m/z) / 392 (quantified m/z) 
# 396 (Quantified m/z)/ 450 (cant do anything) / 608 (m/z) /763 (moverz_quant) / 918 (quantified m/z) ) 


Feng
plasmaall_author    Mass-to-Charge
0      113.020753
1      113.020780
2      115.036473
3      125.015184
4      127.038142
urineall_author    Mass-to-Charge
0       94.041381
1      123.080474
2      125.015218
3      127.012272
4      128.062004
serum_IPO_aligned_Feng_serum_batch1           mz      mzmin      mzmax           rt        rtmin        rtmax  X1  \
0  54.060870  54.060830  54.060887   501.432343   499.808777   505.116608   4   
1  60.080305  60.079990  60.080790  1076.213623  1030.533936  1130.009155  40   
2  60.080234  60.079750  60.080862   992.466309   950.546936  1028.531006  38   
3  60.080354  60.080203  60.080482  1160.527222  1130.977173  1199.477661  39   
4  60.080220  60.079715  60.080873   926.456146   900.350037   949.659668  33   

  isotopes adduct  pcgroup  
0      NaN    NaN       15  
1      NaN    NaN       60  
2      NaN    NaN       58  
3      NaN    NaN       71  
4      NaN    NaN       57  
serum_IPO_aligned_Feng_serum_batch2           mz 

IPO_aligned_MTBLS17_negall_3_30_peakwidth           mz      mzmin      mzmax          rt       rtmin       rtmax   X1  \
0  50.043918  50.027338  50.071893  556.483215  547.329041  564.762573   49   
1  50.219330  50.196077  50.240041  557.721985  549.275757  562.197998   47   
2  50.387911  50.369847  50.418030  510.415512  508.395142  514.762634  110   
3  50.565178  50.545200  50.596117  510.236450  508.456787  520.516602   97   
4  50.743412  50.724239  50.767039  509.769775  508.204041  513.041199   29   

  isotopes adduct  pcgroup  
0      NaN    NaN      245  
1      NaN    NaN     3651  
2      NaN    NaN     4382  
3      NaN    NaN      152  
4      NaN    NaN     3144  
IPO_aligned_MTBLS17_negall_3_90_peakwidth           mz      mzmin      mzmax          rt       rtmin       rtmax   X1  \
0  50.043242  50.026741  50.071893  556.598389  547.329041  564.762573   81   
1  50.219238  50.196077  50.240767  556.855865  546.668213  563.029968   67   
2  50.387572  50.369364  50.41

IPO_aligned_MTBLS19_neg_all_F_R           mz      mzmin      mzmax          rt       rtmin       rtmax   X1  \
0  51.442447  51.425509  51.454683  524.848572  521.046753  530.029724   19   
1  51.621198  51.600598  51.630432  524.337891  519.782715  529.519287   19   
2  59.013611  59.012357  59.014751   27.498774   26.175179   40.366772  179   
3  61.987929  61.986766  61.989038  421.180817  420.296814  422.277710   75   
4  61.987686  61.986895  61.988737  692.693970  692.361328  694.372314   14   

  isotopes adduct  pcgroup  
0      NaN    NaN      763  
1      NaN    NaN       27  
2      NaN    NaN       13  
3      NaN    NaN       21  
4      NaN    NaN      118  
IPO_aligned_MTBLS19_neg_all_R_R           mz      mzmin      mzmax          rt       rtmin       rtmax   X1  \
0  51.442447  51.425509  51.454683  524.848572  521.046753  530.029724   19   
1  51.621198  51.600598  51.630432  524.337891  519.782715  529.519287   19   
2  59.013611  59.012357  59.014751   27.498774   2

m_chronic_hep_b_NEG   database_identifier chemical_formula     smiles inchi  \
0         CHEBI:25627         C18H32O2        NaN   NaN   
1         CHEBI:25633         C18H30O2        NaN   NaN   
2         CHEBI:36036         C20H34O2        NaN   NaN   
3         CHEBI:36036         C20H34O2        NaN   NaN   
4         CHEBI:36033         C20H32O2  OC([*])=O   NaN   

  metabolite_identification  mass_to_charge fragmentation modifications  \
0                 FA (18:2)      279.232950                      [M-H]-   
1            FA (18:3-w3w6)      277.217300                      [M-H]-   
2            FA (20:3-w3w6)      305.248604                      [M-H]-   
3              FA (20:3-w9)      305.248604                      [M-H]-   
4              FA (20:4-w6)      303.232950                      [M-H]-   

   charge  retention_time                                              taxid  \
0     NaN             NaN  http://purl.bioontology.org/ontology/NCBITAXON...   
1     NaN     

IPO_aligned_MTBLS92            mz       mzmin       mzmax          rt       rtmin       rtmax  \
0  300.290330  300.283457  300.298898  193.692322  180.327362  195.552765   
1  301.141869  301.138782  301.145080  146.317993  143.545609  148.368103   
2  301.217148  301.214051  301.231446  446.572052  441.768433  447.334595   
3  301.216760  301.209441  301.226449  432.300262  428.527130  434.868805   
4  301.213863  301.204203  301.224028  225.245377  223.910309  226.307709   

    X1 isotopes adduct  pcgroup  
0  194      NaN    NaN     1969  
1  447  [1][M]+    NaN       93  
2   83  [2][M]+    NaN       33  
3   84      NaN    NaN     1200  
4   47      NaN    NaN     1721  
Author_data    mass_to_charge  retention_time
0        516.5156             390
1        574.5186             390
2        468.3113             158
3        482.3634             190
4        494.3262             162
ST000045
02Feb10-21-r0_ND_II        Mass  Retention Time Ionization mode
0  432.1992        0.716

4      NaN    NaN     1363  
IPO_aligned_ST000046_20120618_pos_c18_CN_MCI           mz      mzmin      mzmax        rt    rtmin    rtmax  npeaks  \
0  56.012201  56.012161  56.012268   42.7310   36.537   50.415       8   
1  56.011859  56.011554  56.012062   78.3495   73.215   79.653       6   
2  56.012179  56.012142  56.012331  819.3240  791.628  867.578       8   
3  56.012233  56.012032  56.012272  751.7265  728.900  766.258       4   
4  56.012175  56.012147  56.012204  698.6840  692.241  705.127       2   

  isotopes adduct  pcgroup  
0      NaN    NaN     7811  
1      NaN    NaN     6573  
2      NaN    NaN     6969  
3      NaN    NaN     7344  
4      NaN    NaN     7723  
IPO_aligned_ST000046_20120618_pos_c18_CN_AD           mz      mzmin      mzmax        rt    rtmin    rtmax  npeaks  \
0  56.012201  56.012161  56.012268   42.7310   36.537   50.415       8   
1  56.011859  56.011554  56.012062   78.3495   73.215   79.653       6   
2  56.012179  56.012142  56.012331  819.3

AN000525_MCD_Control                       Identifier       m/z     RT  \
metabolite_name                                     
Acylcarnitine C10:0  0.58_316.25  316.2478  0.591   
Acylcarnitine C12:0  0.73_344.28  344.2801  0.740   
Acylcarnitine C16:0  1.58_400.34   400.342  1.800   
Acylcarnitine C18:0  2.33_428.37  428.3727  2.544   
Acylcarnitine C18:1  1.73_426.36  426.3578  1.950   

                                       InChi key Adducts            Comments  \
metabolite_name                                                                
Acylcarnitine C10:0  LZOSYCMHQXPBFU-UHFFFAOYSA-N  [M+H]+                 NaN   
Acylcarnitine C12:0  FUJLYHJROOYKRA-QGZVFWFLSA-N  [M+H]+                 NaN   
Acylcarnitine C16:0  XOMRRQXKHMYMOC-OAQYLSRUSA-N  [M+H]+  palmitoylcarnitine   
Acylcarnitine C18:0  FNPHNLNTJNMAEE-UHFFFAOYSA-N  [M+H]+   stearoylcarnitine   
Acylcarnitine C18:1  HITOYGLMAFIRNI-YSESTWPTSA-N  [M+H]+      oleylcarnitine   

                     mz  rt  
metabolite_name 

4        6  
ST000392
AN000628_plasma                  retention index  quantified m/z  KEGG ID  PubChem ID  mz  rt
metabolite_name                                                              
xylose                    543267             103      NaN         NaN NaN NaN
xylitol                   567437             217      NaN         NaN NaN NaN
valine                    313502             144      NaN         NaN NaN NaN
uridine                   856709             224      NaN         NaN NaN NaN
uric acid                 730691             441      NaN         NaN NaN NaN
AN000628_serum                  retention index  quantified m/z  KEGG ID  PubChem ID  mz  rt
metabolite_name                                                              
xylose                    543267             103      NaN         NaN NaN NaN
xylitol                   567437             217      NaN         NaN NaN NaN
valine                    313502             144      NaN         NaN NaN NaN
uridine    

4      NaN              NaN      126  
IPO_aligned_ST000578_C18           mz      mzmin      mzmax         rt      rtmin      rtmax   X1  \
0  85.514413  85.508536  85.515435  37.251535  35.152229  39.531582  165   
1  86.095243  86.094879  86.096328  48.936152  46.349087  56.708641  834   
2  86.516601  86.516380  86.517660  36.775433  34.362762  38.738209  179   
3  87.098618  87.098226  87.099589  50.178110  47.222454  54.529900   67   
4  88.111410  88.110679  88.111739  59.895256  56.598213  64.961174   85   

    isotopes           adduct  pcgroup  
0        NaN   [M+H]+ 84.5038       27  
1    [1][M]+  [M+Na]+ 63.1039        6  
2        NaN              NaN     7077  
3  [1][M+1]+              NaN     2375  
4        NaN              NaN     7210  
ST000608
AN000929                         m/z  retention time  mz  rt
metabolite_name                                    
Cer(d18:0/18:0)  568.563467       59.311302 NaN NaN
Cer(d18:0/20:0)  596.594335       61.853355 NaN NaN
Cer(d18

IPO_aligned_ST000763_untar_pos_Normal Pressures_Borderline Pressures           mz      mzmin      mzmax          rt       rtmin       rtmax   X1  \
0  53.019350  53.019075  53.019919   32.930960   32.641933   34.318851   58   
1  53.520069  53.519630  53.520452   33.349239   32.859509   34.318851   49   
2  54.018299  54.017846  54.018708   33.131647   32.859509   34.545029   44   
3  55.018177  55.017975  55.018593  118.236740  117.389717  119.114922   15   
4  55.054761  55.054292  55.055304  129.585480  128.107773  131.199982  184   

     isotopes adduct  pcgroup  
0    [1][M]2+    NaN       31  
1  [1][M+1]2+    NaN       31  
2  [1][M+2]2+    NaN       31  
3         NaN    NaN     8082  
4         NaN    NaN      101  
IPO_aligned_ST000763_untar_pos_Normal Pressures_LowRisk           mz      mzmin      mzmax          rt       rtmin       rtmax   X1  \
0  53.019350  53.019075  53.019919   32.930960   32.641933   34.318851   58   
1  53.520069  53.519630  53.520452   33.349239   3